In [274]:
from pandas import read_excel
# Importing necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus# sample text for performing tokenization
from lxml import html
import requests
from nltk.tokenize import word_tokenize# Passing the string text into word tokenize for breaking the sentences
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
import unidecode
# faut lancer ca la premiere fois -> nltk.download()

In [275]:
bouffe = read_excel("bouffe.xls")

In [276]:
recipe = []

In [277]:
french_stopwords = set(stopwords.words('french'))
filtre_stopfr =  lambda text: [token for token in text if token.lower() not in french_stopwords]

In [281]:

page = requests.get('https://www.marmiton.org/recettes/recette_sauce-puttanesca_14929.aspx')
# https://www.marmiton.org/recettes/recette_sauce-puttanesca_14929.aspx
tree = html.fromstring(page.content)
products = tree.xpath("//div[contains(@class, 'MuiGrid-root')]")
for product in products[1:]:
    recipe.append(product.text_content())

In [297]:
recipe

["3\xa0cuillères d'huile d'olive ",
 "2\xa0gousses d'ail écrasées",
 '1 piment rouge séché écrasé',
 '2\xa0cuillères de câpres ',
 '500\xa0g de pâtes ',
 "6\xa0filets d'anchois hachés",
 '500\xa0g de tomates pelées , épépinées et concassées',
 "125\xa0g d'olives noires dénoyautées et coupées en 2"]

In [18]:
def normalize_text(text):
    text = unaccented_string = unidecode.unidecode(str(text))    
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    words = filtre_stopfr( word_tokenize(text, language="french") )
    stemmer = FrenchStemmer()
    res = " ".join([stemmer.stem(w) for w in words])
    return res

In [282]:
bouffe["alim_nom_fr"] = bouffe["alim_nom_fr"].apply(normalize_text)

In [20]:
bouffe["alim_ssgrp_nom_fr"] = bouffe["alim_ssgrp_nom_fr"].apply(normalize_text)

In [302]:
set(bouffe["alim_ssssgrp_nom_fr"])

{'-',
 'abats',
 'agneau et mouton',
 'autres desserts',
 'autres fromages et spécialités',
 'autres spécialités charcutières',
 'autres viandes',
 'biscottes et pains grillés',
 'bières et cidres',
 'boissons rafraîchissantes lactées',
 'boissons rafraîchissantes sans alcool',
 'boissons végétales',
 'boissons à reconstituer',
 'bœuf et veau',
 'café, thé, cacao etc. prêts à consommer',
 'cocktails',
 'compotes et assimilés',
 'desserts lactés',
 'desserts végétaux',
 'dinde',
 'fromage fondus',
 'fromages blancs',
 'fromages à pâte molle',
 'fromages à pâte persillée',
 'fromages à pâte pressée',
 'fruits appertisés',
 'fruits crus',
 'fruits et leurs produits de la Martinique',
 'fruits et leurs produits de la Réunion',
 'fruits séchés',
 'gibier',
 'herbes fraîches',
 'herbes séchées',
 'jambons cuits',
 'jambons secs et crus',
 'jus',
 'laits autres que de vache',
 'laits de vache concentrés ou en poudre',
 'laits de vaches liquides (non concentrés)',
 'liqueurs et alcools',
 'lég

In [8]:
descriptions

0                               dessert aliment moyen
1                          salad thon legum appertise
2               salad compose viand poisson appertise
3                        champignon a grecqu appertis
4                         salad pomm terr fait maison
                            ...                      
3181           dessert lact infantil natur sucr fruit
3182    cereal instantane poudr a reconstitu 4 6 mois
3183      cereal instantane poudr a reconstitu 6 mois
3184                                      biscuit beb
3185                                poudr cacaote beb
Name: alim_nom_fr, Length: 3186, dtype: object

In [256]:
def words_in_column(bouffe, prod, col):
    mask = pd.Series([True for i in range(bouffe.shape[0])])
    for word in prod.split(" "):
        mask = mask & bouffe[col].str.contains(word)
    return bouffe.loc[mask]

In [290]:
def food_matcher(nom_aliment):
    prod = normalize_text(nom_aliment)
    products_fit = words_in_column(bouffe, prod, 'alim_nom_fr')
    print(f"size products {products_fit.shape[0]}")
    shorter_description_sample = products_fit["alim_nom_fr"].apply(len).sort_values()[:10].index
    print(shorter_description_sample)
    ten_shorter_des_match = bouffe.loc[shorter_description_sample]
    print(ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts())
    best_cat = ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts().index[0]
    if best_cat == "nan":
        best_cat = ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts().index[1]
    ten_shorter_des_match = ten_shorter_des_match.loc[ten_shorter_des_match["alim_ssgrp_nom_fr"]==best_cat]
    best_bet = ten_shorter_des_match.iloc[0]
    print(f"Best match : {best_bet['alim_nom_fr']} \nBest cat : {best_cat}")
    return best_bet

In [296]:
food_matcher("oignon")

size products 21
Int64Index([352, 598, 481, 3016, 440, 441, 437, 248, 2973, 119], dtype='int64')
légumes                            6
condiments                         1
pizzas, tartes et crêpes salées    1
sauces                             1
plats composés                     1
Name: alim_ssgrp_nom_fr, dtype: int64
Best match : oignon cru 
Best cat : légumes


alim_grp_code                                                                              2
alim_ssgrp_code                                                                          201
alim_ssssgrp_code                                                                      20101
alim_grp_nom_fr                                  fruits, légumes, légumineuses et oléagineux
alim_ssgrp_nom_fr                                                                    légumes
                                                                    ...                     
Vitamine B3 ou PP ou Niacine (mg/100 g)                                               < 0,05
Vitamine B5 ou Acide pantothénique (mg/100 g)                                           0,15
Vitamine B6 (mg/100 g)                                                                   0,1
Vitamine B9 ou Folates totaux (µg/100 g)                                                29,6
Vitamine B12 (µg/100 g)                                               